In [ ]:
import tensorflow as tf

print(tf.__version__)

2.2.0


In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# ————————————————
# 版权声明：本文为CSDN博主「LCCFlccf」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/LCCFlccf/java/article/details/89302730

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
# 指定Google Drive云端硬盘的根目录，名为drive
# !mkdir -p drive
# !google-drive-ocamlfuse drive

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls /content/drive/'My Drive'
# !ls /content/drive/'My Drive'/audio_classifier_tutorial/UrbanSound8K
# !top

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 audio_classifier_tutorial		  'Sleep Monitor_app_v121.gsheet'
 bq-results-20200331-154548-i5vvaponde5c  'Sleep Monitor Feedback.gsheet'
 bq-results-20200331-154548-qpo4cr21f6xp  'SM_Music License Agreement.gdoc'
'Colab Notebooks'			   SM_订阅页面翻译v122.gsheet
'DrinkWR2020  ROI.gsheet'		  'Untitled spreadsheet.gsheet'
'DWR 2020-6月.gsheet'			   UrbanSound8K_README.txt
'DWR 2020-7月.gsheet'			   广告数据分析表.gsheet
 FREESOUNDCREDITS.txt			   美国.gsheet
 LOGO_512.png


In [ ]:
# https://blog.doiduoyi.com/articles/1587654005620.html?

import os

import librosa

# from numba.decorators import jit as optional_jit


def get_data_list(audio_path, list_path):
    sound_sum = 0
    audios = os.listdir(audio_path)

    f_train = open(os.path.join(list_path, 'train_list.txt'), 'w')
    f_test = open(os.path.join(list_path, 'test_list.txt'), 'w')

    for i in range(len(audios)):
        sounds = os.listdir(os.path.join(audio_path, audios[i]))
        for sound in sounds:
            sound_path = os.path.join(audio_path, audios[i], sound)
            print("get_data_list, sound_path = %s " % sound_path)

            try:
                t = librosa.get_duration(filename=sound_path)
                print("get_data_list, get_duration = %s " % t)
                # [可能需要修改参数] 过滤小于2.1秒的音频
                if t >= 2.1:
                    if sound_sum % 100 == 0:
                        f_test.write('%s\t%d\n' % (sound_path, i))
                    else:
                        f_train.write('%s\t%d\n' % (sound_path, i))
                    sound_sum += 1
            except:
                print("get_data_list, except sound_path = %s" % sound_path)

        print("get_data_list, Audio：%d/%d" % (i + 1, len(audios)))

    f_test.close()
    f_train.close()


if __name__ == '__main__':
    get_data_list('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/audio', '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata')

In [ ]:
# create_tfrecord.py

import tensorflow as tf
import librosa
import random
import numpy as np
from tqdm import tqdm

# 获取浮点数组
def _float_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


# 获取整型数据
def _int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


# 把数据添加到TFRecord中
def data_example(data, label):
    feature = {
        'data': _float_feature(data),
        'label': _int64_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


# 开始创建tfrecord数据
def create_data_tfrecord(data_list_path, save_path):
    with open(data_list_path, 'r') as f:
        data = f.readlines()
    with tf.io.TFRecordWriter(save_path) as writer:
        for d in tqdm(data):
            try:
                path, label = d.replace('\n', '').split('\t')
                wav, sr = librosa.load(path, sr=16000)
                intervals = librosa.effects.split(wav, top_db=20)
                wav_output = []
                # [可能需要修改参数] 音频长度 16000 * 秒数
                wav_len = int(16000 * 2.04)
                for sliced in intervals:
                    wav_output.extend(wav[sliced[0]:sliced[1]])
                for i in range(5):
                    # 裁剪过长的音频，过短的补0
                    if len(wav_output) > wav_len:
                        l = len(wav_output) - wav_len
                        r = random.randint(0, l)
                        wav_output = wav_output[r:wav_len + r]
                    else:
                        wav_output.extend(np.zeros(shape=[wav_len - len(wav_output)], dtype=np.float32))
                    wav_output = np.array(wav_output)
                    # 转成梅尔频谱
                    ps = librosa.feature.melspectrogram(y=wav_output, sr=sr, hop_length=256).reshape(-1).tolist()
                    # [可能需要修改参数] 梅尔频谱shape ，librosa.feature.melspectrogram(y=wav_output, sr=sr, hop_length=256).shape
                    if len(ps) != 128 * 128: continue
                    tf_example = data_example(ps, int(label))
                    writer.write(tf_example.SerializeToString())
                    if len(wav_output) <= wav_len:
                        break
            except Exception as e:
                print(e)


if __name__ == '__main__':
    create_data_tfrecord('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata/train_list.txt', '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata/train.tfrecord')
    create_data_tfrecord('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata/test_list.txt', '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata/test.tfrecord')

100%|██████████| 78/78 [01:14<00:00,  1.05it/s]


In [ ]:
# train_data.py

import tensorflow as tf
import numpy as np

class_dim = 10
EPOCHS = 100
BATCH_SIZE=32
# EPOCHS = 1
# BATCH_SIZE=16

tf.compat.v1.enable_eager_execution() # WARNING:tensorflow:From E:/workspace/python/doiduoyi/src/train_data.py:8: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.
#tf.enable_eager_execution() # https://github.com/tensorflow/datasets/issues/233

model = tf.keras.models.Sequential([
    tf.keras.applications.ResNet50V2(include_top=False, weights=None, input_shape=(128, None, 1)),
    tf.keras.layers.ActivityRegularization(l2=0.5),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(units=class_dim, activation=tf.nn.softmax)
])

model.summary()


# 定义优化方法
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

##########################################################

def _parse_data_function(example):
    # [可能需要修改参数】 设置的梅尔频谱的shape相乘的值
    data_feature_description = {
        'data': tf.io.FixedLenFeature([16384], tf.float32),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    return tf.io.parse_single_example(example, data_feature_description)


def train_reader_tfrecord(data_path, num_epochs, batch_size):
    raw_dataset = tf.data.TFRecordDataset(data_path)
    train_dataset = raw_dataset.map(_parse_data_function)
    train_dataset = train_dataset.shuffle(buffer_size=1000) \
        .repeat(count=num_epochs) \
        .batch(batch_size=batch_size) \
        .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset


def test_reader_tfrecord(data_path, batch_size):
    raw_dataset = tf.data.TFRecordDataset(data_path)
    test_dataset = raw_dataset.map(_parse_data_function)
    test_dataset = test_dataset.batch(batch_size=batch_size)
    return test_dataset
##########################################################


# train_dataset = reader.train_reader_tfrecord('F:/Downloads/UrbanSound8K/metadata//train.tfrecord', EPOCHS, batch_size=BATCH_SIZE)
# test_dataset = reader.test_reader_tfrecord('F:/Downloads/UrbanSound8K/metadata//test.tfrecord', batch_size=BATCH_SIZE)

train_dataset = train_reader_tfrecord('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata//train.tfrecord', EPOCHS, batch_size=BATCH_SIZE)
test_dataset = test_reader_tfrecord('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/metadata//test.tfrecord', batch_size=BATCH_SIZE)


##########################################################

for batch_id, data in enumerate(train_dataset):
    # [可能需要修改参数】 设置的梅尔频谱的shape
    sounds = data['data'].numpy().reshape((-1, 128, 128, 1))
    labels = data['label']
    # 执行训练
    with tf.GradientTape() as tape:
        predictions = model(sounds)
        # 获取损失值
        train_loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
        train_loss = tf.reduce_mean(train_loss)
        # 获取准确率
        train_accuracy = tf.keras.metrics.sparse_categorical_accuracy(labels, predictions)
        train_accuracy = np.sum(train_accuracy.numpy()) / len(train_accuracy.numpy())

    # 更新梯度
    gradients = tape.gradient(train_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # if batch_id % 20 == 0:
    #     print("Batch %d, Loss %f, Accuracy %f" % (batch_id, train_loss.numpy(), train_accuracy))
    print("Batch %d, Loss %f, Accuracy %f" % (batch_id, train_loss.numpy(), train_accuracy))

    if batch_id % 200 == 0 and batch_id != 0:
        test_losses = list()
        test_accuracies = list()
        for d in test_dataset:
            try:
                print("Batch %d" % (d))
            except:
                print("Batch except")

            # [可能需要修改参数】 设置的梅尔频谱的shape
            test_sounds = d['data'].numpy().reshape((-1, 128, 128, 1))
            test_labels = d['label']

            test_result = model(test_sounds)
            # 获取损失值
            test_loss = tf.keras.losses.sparse_categorical_crossentropy(test_labels, test_result)
            test_loss = tf.reduce_mean(test_loss)
            test_losses.append(test_loss)
            # 获取准确率
            test_accuracy = tf.keras.metrics.sparse_categorical_accuracy(test_labels, test_result)
            test_accuracy = np.sum(test_accuracy.numpy()) / len(test_accuracy.numpy())
            test_accuracies.append(test_accuracy)

        print('=================================================')
        print("Test, Loss %f, Accuracy %f" % (
            sum(test_losses) / len(test_losses), sum(test_accuracies) / len(test_accuracies)))
        print('=================================================')

        # 保存模型
        # model.save(filepath='F:/Downloads/UrbanSound8K/models/resnet50.h5')
        model.save(filepath='/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/resnet50.h5')
##########################################################

流式输出内容被截断，只能显示最后 5000 行内容。
Batch 18998, Loss 1.453483, Accuracy 0.531250
Batch 18999, Loss 1.169668, Accuracy 0.500000
Batch 19000, Loss 1.337126, Accuracy 0.468750
Batch except
Batch except
Batch except
Test, Loss 5.117010, Accuracy 0.083333
Batch 19001, Loss 1.729754, Accuracy 0.375000
Batch 19002, Loss 1.508246, Accuracy 0.500000
Batch 19003, Loss 1.983719, Accuracy 0.375000
Batch 19004, Loss 1.634984, Accuracy 0.531250
Batch 19005, Loss 1.676693, Accuracy 0.406250
Batch 19006, Loss 1.786031, Accuracy 0.375000
Batch 19007, Loss 1.558875, Accuracy 0.562500
Batch 19008, Loss 1.629131, Accuracy 0.437500
Batch 19009, Loss 1.707987, Accuracy 0.312500
Batch 19010, Loss 1.506931, Accuracy 0.406250
Batch 19011, Loss 1.475863, Accuracy 0.468750
Batch 19012, Loss 1.325006, Accuracy 0.531250
Batch 19013, Loss 1.449077, Accuracy 0.406250
Batch 19014, Loss 1.655189, Accuracy 0.406250
Batch 19015, Loss 1.421569, Accuracy 0.343750
Batch 19016, Loss 1.780589, Accuracy 0.250000
Batch 19017, Loss 1.8

In [ ]:
!ls -la '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/'

total 369623
drwx------ 2 root root     4096 Jun  3 08:23  assets
-rw------- 1 root root 94621224 Jun  4 01:55 'Copy of github_yeyupiaoling_resnet50.h5'
-rw------- 1 root root 94621112 Jun  4 01:55 'Copy of resnet50.h5'
-rw------- 1 root root 94621224 May 29 04:37  github_yeyupiaoling_resnet50.h5
-rw------- 1 root root 94621112 Jun 17 03:26  resnet50.h5
drwx------ 2 root root     4096 Jun  3 08:23  variables


In [ ]:
# predict.py


import librosa
import numpy as np
import tensorflow as tf
import os

# model_path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5' # yeyupiaoling作者的模型, 准确率高
model_path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/resnet50.h5' # 自己训练的模型，准确率低
model = tf.keras.models.load_model(model_path)

# 读取音频数据
def load_data(data_path):
    wav, sr = librosa.load(data_path, sr=16000)
    intervals = librosa.effects.split(wav, top_db=20)
    wav_output = []
    for sliced in intervals:
        wav_output.extend(wav[sliced[0]:sliced[1]])
    assert len(wav_output) >= 8000, "有效音频小于0.5s"
    wav_output = np.array(wav_output)
    ps = librosa.feature.melspectrogram(y=wav_output, sr=sr, hop_length=256).astype(np.float32)
    ps = ps[np.newaxis, ..., np.newaxis]
    return ps


def infer(audio_path):
    print('infer：audio_path %s' % (audio_path))

    data = load_data(audio_path)
    result = model.predict(data)
    print('infer：result %s' % (result))

    max = np.max(result, axis=1)
    print('infer：max %s' % (max))

    index = np.argmax(result, axis=1)
    print('infer：index %s' % (index))

    # predict_dataset = tf.convert_to_tensor(result)
    # print('infer：predict_dataset %s' % (predict_dataset))

    # predictions = model(predict_dataset)
    # print('infer：predictions %s' % (predictions))

    # class_idx = tf.argmax(predictions).numpy()
    # p = tf.nn.softmax(predictions)[class_idx]
    # print('infer：class_idx %s' % (class_idx))

    lab = tf.argmax(result, 1, output_type=tf.int32)
    print('infer：argmax %s' % (lab))

    # argmax2 = tf.argmax(predict_dataset, 1, output_type=tf.float32)
    # print('infer：argmax2 %s' % (argmax2))

    return lab



def infer2(audio_path):
    print('infer2：audio_path %s' % (audio_path))

    data = load_data(audio_path)
    result = model.predict(data)
    print('infer2：result %s' % (result))

    max = np.max(result, axis=1)
    print('infer2：max %s' % (max))

    index = np.argmax(result, axis=1)
    print('infer2：index %s' % (index))

    labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']
    #          0                     1            2                      3            4              5                  6             7               8         9
    print('infer2：labels %s' % labels[int(index)])


    return index


if __name__ == '__main__':

    print('model_path %s ' % (model_path))

    # 要预测的音频文件
    # path = 'F:/Downloads/UrbanSound8K/predict/fold5.wav'fold8-4918-3-0-0.wav
    # path = 'F:/Downloads/UrbanSound8K/predict/fold8-4918-3-0-0.wav'
    # path = 'F:/Downloads/UrbanSound8K/predict/fold8-17009-2-0-4.wav'
    # path = 'F:/Downloads/UrbanSound8K/predict/fold8-17009-2-0-10.wav'

    # path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict/server'
    path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict'
    wav_list = os.listdir(path)
    for wav in wav_list:
        print('音频：########################################################')
        uri = os.path.join(path, wav)
        print('音频：uri %s' % (uri))

        try:

            # label = infer(uri)
            # labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling', 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']
            #          0                     1                      2             3            4                   5             6               7             8         9
            index = infer2(uri)
            # print('音频：%s 的预测结果标签为：index %d' % index)
            # print('音频：%s 的预测结果标签为：%s' % (path, labels[index]))

        except:
            print('音频：except')




model_path /content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/resnet50.h5 
音频：########################################################
音频：uri /content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict/7061-6-0-0.wav
infer2：audio_path /content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict/7061-6-0-0.wav
infer2：result [[0.02111227 0.01462406 0.0033954  0.00572845 0.02343955 0.04079945
  0.07555167 0.35237265 0.3899884  0.07298808]]
infer2：max [0.3899884]
infer2：index [8]
infer2：labels siren
音频：########################################################
音频：uri /content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict/fold10-7965-3-11-0.wav
infer2：audio_path /content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/predict/fold10-7965-3-11-0.wav
infer2：result [[0.03738586 0.02860939 0.00855223 0.0106274  0.03207279 0.05144677
  0.08822538 0.33424968 0.3269789  0.08185166]]
infer2：max [0.33424968]
infer2：index [7]
infer2：labels j

In [ ]:
# tensorflow_lite_convertor.py

# https://www.tensorflow.org/lite/convert/python_api?hl=zh-cn


# # -3
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/My Drive/audio_classifier_tutorial/inception_dec_2015')
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)


# # -2
# from tensorflow.contrib import lite
# converter = lite.TFLiteConverter.from_saved_model('/content/drive/My Drive/audio_classifier_tutorial/inception_dec_2015/tensorflow_inception_graph.pb')
# tfmodel = converter.convert()
# open ("model.tflite" , "wb") .write(tfmodel)

# import tensorflow as tf

# # https://www.tensorflow.org/tutorials/keras/save_and_load
# # 重新创建完全相同的模型，包括其权重和优化程序
# new_model = tf.keras.models.load_model('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5')
# # 显示网络结构
# new_model.summary()

# converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
# tflite_model = converter.convert()


# -1
# !tflite_convert --output_file='/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/foo.tflite' --keras_model_file='/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5'

# 0
# import tensorflow as tf

# # 建立一个简单的模型。
# # root = tf.train.Checkpoint()
# # root.v1 = tf.Variable(3.)
# # root.v2 = tf.Variable(2.)
# # root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# # 保存模型。
# export_dir = "/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models"
# # input_data = tf.constant(1., shape=[1, 1])
# # to_save = root.f.get_concrete_function(input_data)
# # tf.saved_model.save(root, export_dir, to_save)

# # 转换模型。
# converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# tflite_model = converter.convert()


# 1
# import tensorflow as tf

# # 建立一个简单的模型。
# root = tf.train.Checkpoint()
# root.v1 = tf.Variable(3.)
# root.v2 = tf.Variable(2.)
# root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# # 保存模型。
# model_path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models' # yeyupiaoling作者的模型, 准确率高
# export_dir = model_path
# # input_data = tf.constant(1., shape=[1, 1])
# # to_save = root.f.get_concrete_function(input_data)
# # tf.saved_model.save(root, export_dir, to_save)

# # 转换模型。
# model = tf.saved_model.load(export_dir)
# concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
# concrete_func.inputs[0].set_shape([1, 256, 256, 3])
# converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

# # converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# tflite_model = converter.convert()


# 2
# import tensorflow as tf
#
# # 创建一个简单的 Keras 模型。
# x = [-1, 0, 1, 2, 3, 4]
# y = [-3, -1, 1, 3, 5, 7]
#
# model = tf.keras.models.Sequential(
#     [tf.keras.layers.Dense(units=1, input_shape=[1])])
# model.compile(optimizer='sgd', loss='mean_squared_error')
# model.fit(x, y, epochs=50)
#
# # 转换模型。
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()



# 3
# import tensorflow as tf

# # 建立一个模型。
# root = tf.train.Checkpoint()
# root.v1 = tf.Variable(3.)
# root.v2 = tf.Variable(2.)
# root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# # 生成 concrete function。
# input_data = tf.constant(1., shape=[1, 1])
# concrete_func = root.f.get_concrete_function(input_data)

# # 转换模型。
# #
# # `from_concrete_function` 的传入参数被设计为一个一个 concrete function 的列表，然而
# # 现阶段仅支持每次调用时仅接受一个concrete function。
# # 同时转换多个concrete function的功能正在开发中。
# converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
# tflite_model = converter.convert()

OSError: ignored

In [ ]:
# https://www.cnblogs.com/chenzhen0530/p/10685944.html

#*-coding:utf-8-*

"""
将keras的.h5的模型文件，转换成TensorFlow的pb文件
"""
# ==========================================================

# from keras.models import load_model
import keras
import tensorflow as tf
import os
from keras import backend


def h5_to_pb(h5_model, output_dir, model_name, out_prefix="output_", log_tensorboard=True):
    """.h5模型文件转换成pb模型文件
    Argument:
        h5_model: str
            .h5模型文件
        output_dir: str
            pb模型文件保存路径
        model_name: str
            pb模型文件名称
        out_prefix: str
            根据训练，需要修改
        log_tensorboard: bool
            是否生成日志文件
    Return:
        pb模型文件
    """
    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)
    out_nodes = []
    for i in range(len(h5_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(h5_model.output[i], out_prefix + str(i + 1))
    sess = backend.get_session()

    from tensorflow.python.framework import graph_util, graph_io
    # 写入pb模型文件
    init_graph = sess.graph.as_graph_def()
    main_graph = graph_util.convert_variables_to_constants(sess, init_graph, out_nodes)
    graph_io.write_graph(main_graph, output_dir, name=model_name, as_text=False)
    # 输出日志文件
    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard
        import_pb_to_tensorboard.import_to_tensorboard(os.path.join(output_dir, model_name), output_dir)


if __name__ == '__main__':
    #  .h模型文件路径参数
    input_path = '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/'
    weight_file = 'github_yeyupiaoling_resnet50.h5'
    weight_file_path = os.path.join(input_path, weight_file)
    output_graph_name = weight_file[:-3] + '.pb'

    #  pb模型文件输出输出路径
    output_dir = os.path.join(os.getcwd(), "/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/")

    #  加载模型
    h5_model = load_model(weight_file_path)
    h5_to_pb(h5_model, output_dir=output_dir, model_name=output_graph_name)
    print('Finished')

ValueError: ignored

In [ ]:
# https://github.com/amir-abdi/keras_to_tensorflow/blob/master/keras_to_tensorflow.py

#!/usr/bin/env python
"""
Copyright (c) 2019, by the Authors: Amir H. Abdi
This script is freely available under the MIT Public License.
Please see the License file in the root for details.
The following code snippet will convert the keras model files
to the freezed .pb tensorflow weight file. The resultant TensorFlow model
holds both the model architecture and its associated weights.
"""

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from pathlib import Path
from absl import app
from absl import flags
from absl import logging
import keras
from keras import backend as K
from keras.models import model_from_json, model_from_yaml

K.set_learning_phase(0)
FLAGS = flags.FLAGS

flags.DEFINE_string('input_model', None, 'Path to the input model.')
flags.DEFINE_string('input_model_json', None, 'Path to the input model '
                                              'architecture in json format.')
flags.DEFINE_string('input_model_yaml', None, 'Path to the input model '
                                              'architecture in yaml format.')
flags.DEFINE_string('output_model', None, 'Path where the converted model will '
                                          'be stored.')
flags.DEFINE_boolean('save_graph_def', False,
                     'Whether to save the graphdef.pbtxt file which contains '
                     'the graph definition in ASCII format.')
flags.DEFINE_string('output_nodes_prefix', None,
                    'If set, the output nodes will be renamed to '
                    '`output_nodes_prefix`+i, where `i` will numerate the '
                    'number of of output nodes of the network.')
flags.DEFINE_boolean('quantize', False,
                     'If set, the resultant TensorFlow graph weights will be '
                     'converted from float into eight-bit equivalents. See '
                     'documentation here: '
                     'https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms')
flags.DEFINE_boolean('channels_first', False,
                     'Whether channels are the first dimension of a tensor. '
                     'The default is TensorFlow behaviour where channels are '
                     'the last dimension.')
flags.DEFINE_boolean('output_meta_ckpt', False,
                     'If set to True, exports the model as .meta, .index, and '
                     '.data files, with a checkpoint file. These can be later '
                     'loaded in TensorFlow to continue training.')

flags.mark_flag_as_required('input_model')
flags.mark_flag_as_required('output_model')


def load_model(input_model_path, input_json_path=None, input_yaml_path=None):
    if not Path(input_model_path).exists():
        raise FileNotFoundError(
            'Model file `{}` does not exist.'.format(input_model_path))
    try:
        model = keras.models.load_model(input_model_path)
        return model
    except FileNotFoundError as err:
        logging.error('Input mode file (%s) does not exist.', FLAGS.input_model)
        raise err
    except ValueError as wrong_file_err:
        if input_json_path:
            if not Path(input_json_path).exists():
                raise FileNotFoundError(
                    'Model description json file `{}` does not exist.'.format(
                        input_json_path))
            try:
                model = model_from_json(open(str(input_json_path)).read())
                model.load_weights(input_model_path)
                return model
            except Exception as err:
                logging.error("Couldn't load model from json.")
                raise err
        elif input_yaml_path:
            if not Path(input_yaml_path).exists():
                raise FileNotFoundError(
                    'Model description yaml file `{}` does not exist.'.format(
                        input_yaml_path))
            try:
                model = model_from_yaml(open(str(input_yaml_path)).read())
                model.load_weights(input_model_path)
                return model
            except Exception as err:
                logging.error("Couldn't load model from yaml.")
                raise err
        else:
            logging.error(
                'Input file specified only holds the weights, and not '
                'the model definition. Save the model using '
                'model.save(filename.h5) which will contain the network '
                'architecture as well as its weights. '
                'If the model is saved using the '
                'model.save_weights(filename) function, either '
                'input_model_json or input_model_yaml flags should be set to '
                'to import the network architecture prior to loading the '
                'weights. \n'
                'Check the keras documentation for more details '
                '(https://keras.io/getting-started/faq/)')
            raise wrong_file_err


def main(args):
    # If output_model path is relative and in cwd, make it absolute from root
    output_model = FLAGS.output_model
    if str(Path(output_model).parent) == '.':
        output_model = str((Path.cwd() / output_model))

    output_fld = Path(output_model).parent
    output_model_name = Path(output_model).name
    output_model_stem = Path(output_model).stem
    output_model_pbtxt_name = output_model_stem + '.pbtxt'

    # Create output directory if it does not exist
    Path(output_model).parent.mkdir(parents=True, exist_ok=True)

    if FLAGS.channels_first:
        K.set_image_data_format('channels_first')
    else:
        K.set_image_data_format('channels_last')

    model = load_model(FLAGS.input_model, FLAGS.input_model_json, FLAGS.input_model_yaml)

    # TODO(amirabdi): Support networks with multiple inputs
    orig_output_node_names = [node.op.name for node in model.outputs]
    if FLAGS.output_nodes_prefix:
        num_output = len(orig_output_node_names)
        pred = [None] * num_output
        converted_output_node_names = [None] * num_output

        # Create dummy tf nodes to rename output
        for i in range(num_output):
            converted_output_node_names[i] = '{}{}'.format(
                FLAGS.output_nodes_prefix, i)
            pred[i] = tf.identity(model.outputs[i],
                                  name=converted_output_node_names[i])
    else:
        converted_output_node_names = orig_output_node_names
    logging.info('Converted output node names are: %s',
                 str(converted_output_node_names))

    sess = K.get_session()
    if FLAGS.output_meta_ckpt:
        saver = tf.train.Saver()
        saver.save(sess, str(output_fld / output_model_stem))

    if FLAGS.save_graph_def:
        tf.train.write_graph(sess.graph.as_graph_def(), str(output_fld),
                             output_model_pbtxt_name, as_text=True)
        logging.info('Saved the graph definition in ascii format at %s',
                     str(Path(output_fld) / output_model_pbtxt_name))

    if FLAGS.quantize:
        from tensorflow.tools.graph_transforms import TransformGraph
        transforms = ["quantize_weights", "quantize_nodes"]
        transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], converted_output_node_names, transforms)
        constant_graph = graph_util.convert_variables_to_constants(
            sess,
            transformed_graph_def,
            converted_output_node_names)
    else:
        constant_graph = graph_util.convert_variables_to_constants(
            sess,
            sess.graph.as_graph_def(),
            converted_output_node_names)

    graph_io.write_graph(constant_graph, str(output_fld), output_model_name, as_text=False)
    logging.info('Saved the freezed graph at %s', str(Path(output_fld) / output_model_name))


if __name__ == "__main__":
    app.run(main)


# main('/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5'， '/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5.pb')

DuplicateFlagError: ignored

In [ ]:

# https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph



from keras import backend as K

# Create, compile and train model...

# frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])
frozen_graph = freeze_session(tf.keras.backend.get_session(), output_names=[out.op.name for out in model.outputs])


# Then you can write the graph to a file as usual with tf.train.write_graph:

tf.train.write_graph(frozen_graph, "/content/drive/My Drive/audio_classifier_tutorial/UrbanSound8K/models/github_yeyupiaoling_resnet50.h5", "my_model.pb", as_text=False)

AttributeError: ignored

In [ ]:
# https://stackoverflow.com/questions/55303156/keras-h5-to-tensorflow-serving-in-2019

from tensorflow import keras
model = keras.models.load_model('/content/drive/My Drive/audio_classifier_tutorial/inception_dec_2015/models/github_yeyupiaoling_resnet50.h5')

keras.experimental.export_saved_model(model, '/content/drive/My Drive/audio_classifier_tutorial/inception_dec_2015/tensorflow_inception_graph.pb')

W0608 03:30:55.979692 140202015917952 hdf5_format.py:187] No training configuration found in the save file, so the model was *not* compiled. Compile it manually.


AttributeError: ignored

In [1]:
!pwd

/content


In [2]:
!ls

sample_data
